## Question #1 Elevation - Clean and gather data
* This notebook inspects and cleans the original data (2019boston_marathon_runners.csv).
* After the original data is cleaned, the Google Geocoding API is used to collect the latitudes & longitudes of each runner's resident city.
* Then the Google Maps Elevation API is used to collect the elevation using the coordinates.
* The cleaned & collected data is stored into a new csv file to use for analysis in a separate notebook.

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json

# Google API Key
from config import g_key

In [2]:
# create dataframe
boston2019_df = pd.read_csv("Resources/2019boston_marathon_runners.csv")
boston2019_df.head()

,BibNumber,FullName,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,Zip,CountryOfResAbbrev,CountryOfResName,CountryOfCtzAbbrev,CountryOfCtzName,OfficialTime,RankOverall,RankOverGender,RankOverDivision,EventGroup,SubGroupLabel,SubGroup
0,2,Lawrence Cherono,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:07:57,1,1,1,Runners,NaN,NaN
1,6,Lelisa Desisa,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,NaN,ETH,Ethiopia,ETH,Ethiopia,2:07:59,2,2,2,Runners,NaN,NaN
2,7,Kenneth Kipkemoi,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:07,3,3,3,Runners,NaN,NaN
3,8,Felix Kandie,"Kandie, Felix",32,M,Iten,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:54,4,4,4,Runners,NaN,NaN
4,11,Geoffrey Kirui,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:55,5,5,5,Runners,NaN,NaN


In [3]:
# inspect data
boston2019_df.count()

BibNumber             26656
FullName              26656
SortName              26656
AgeOnRaceDay          26656
Gender                26656
City                  26655
StateAbbrev           21604
StateName             21603
Zip                   26580
CountryOfResAbbrev    26656
CountryOfResName      26656
CountryOfCtzAbbrev    26656
CountryOfCtzName      26656
OfficialTime          26656
RankOverall           26656
RankOverGender        26656
RankOverDivision      26656
EventGroup            26656
SubGroupLabel           114
SubGroup                113
dtype: int64

In [36]:
# inspect each column
test = boston2019_df.groupby("SubGroupLabel")
test["SubGroupLabel"].count()

SubGroupLabel
DUO         9
MI         55
Runners     1
VI         49
Name: SubGroupLabel, dtype: int64

In [5]:
# inspect unusual results - this will get removed in next step because SubGroupLabel isn't null
test_df = boston2019_df.loc[boston2019_df["Gender"] == "62"]
test_df

,BibNumber,FullName,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,Zip,CountryOfResAbbrev,CountryOfResName,CountryOfCtzAbbrev,CountryOfCtzName,OfficialTime,RankOverall,RankOverGender,RankOverDivision,EventGroup,SubGroupLabel,SubGroup
26368,28913,"Pamela OConnell""",OConnell,"Pamela""",62,F,Leighton Buzzard,NaN,NaN,LU7 4UF,GBR,United Kingdom,GBR,United Kingdom,5:46:30,26143,11695,414,Runners,NaN


In [6]:
# clean data by removing runners in a sub group & who are missing their city
clean2019_df = boston2019_df.loc[boston2019_df["SubGroupLabel"].isnull() & boston2019_df["City"].notnull()]

In [7]:
clean2019_df.count()

BibNumber             26541
FullName              26541
SortName              26541
AgeOnRaceDay          26541
Gender                26541
City                  26541
StateAbbrev           21501
StateName             21501
Zip                   26466
CountryOfResAbbrev    26541
CountryOfResName      26541
CountryOfCtzAbbrev    26541
CountryOfCtzName      26541
OfficialTime          26541
RankOverall           26541
RankOverGender        26541
RankOverDivision      26541
EventGroup            26541
SubGroupLabel             0
SubGroup                  0
dtype: int64

In [8]:
# double-check count
subgrouplabel = clean2019_df.groupby("Gender")
subgrouplabel["Gender"].count()

Gender
F    11946
M    14595
Name: Gender, dtype: int64

In [9]:
# display clean dataframe
clean2019_df

,BibNumber,FullName,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,Zip,CountryOfResAbbrev,CountryOfResName,CountryOfCtzAbbrev,CountryOfCtzName,OfficialTime,RankOverall,RankOverGender,RankOverDivision,EventGroup,SubGroupLabel,SubGroup
0,2,Lawrence Cherono,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:07:57,1,1,1,Runners,NaN,NaN
1,6,Lelisa Desisa,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,NaN,ETH,Ethiopia,ETH,Ethiopia,2:07:59,2,2,2,Runners,NaN,NaN
2,7,Kenneth Kipkemoi,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:07,3,3,3,Runners,NaN,NaN
3,8,Felix Kandie,"Kandie, Felix",32,M,Iten,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:54,4,4,4,Runners,NaN,NaN
4,11,Geoffrey Kirui,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:55,5,5,5,Runners,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26648,30798,Mackenzie Shubert,"Shubert, Mackenzie",28,F,Malden,MA,Massachusetts,02148,USA,United States of America,USA,United States of America,6:16:56,26636,11975,5658,Runners,NaN,NaN
26649,25235,Nikki Mcsweeney,"Mcsweeney, Nikki",33,F,Natick,MA,Massachusetts,01760,USA,United States of America,USA,United States of America,6:18:13,26637,11976,5659,Runners,NaN,NaN
26650,25616,Debbie Genest,"Genest, Debbie",44,F,Harrisville,RI,Rhode Island,02830,USA,United States of America,USA,United States of America,6:21:19,26640,11977,1840,Runners,NaN,NaN
26651,23316,Denise Lewandowski,"Lewandowski, Denise",50,F,Boiling Springs,PA,Pennsylvania,17007,USA,United States of America,USA,United States of America,6:22:27,26641,11978,1199,Runners,NaN,NaN


In [10]:
# get column names
clean2019_df.columns

Index(['BibNumber', 'FullName', 'SortName', 'AgeOnRaceDay', 'Gender', 'City',
       'StateAbbrev', 'StateName', 'Zip', 'CountryOfResAbbrev',
       'CountryOfResName', 'CountryOfCtzAbbrev', 'CountryOfCtzName',
       'OfficialTime', 'RankOverall', 'RankOverGender', 'RankOverDivision',
       'EventGroup', 'SubGroupLabel', 'SubGroup'],
      dtype='object')

In [11]:
# create dataframe with only necessary columns
bos2019_df = clean2019_df.loc[:, ['BibNumber', 'SortName', 'AgeOnRaceDay', 'Gender', 'City',
       'StateAbbrev', 'StateName', 'CountryOfResAbbrev','CountryOfResName',
       'OfficialTime', 'RankOverall', 'RankOverGender']]
bos2019_df.count()

BibNumber             26541
SortName              26541
AgeOnRaceDay          26541
Gender                26541
City                  26541
StateAbbrev           21501
StateName             21501
CountryOfResAbbrev    26541
CountryOfResName      26541
OfficialTime          26541
RankOverall           26541
RankOverGender        26541
dtype: int64

In [12]:
# display new dataframe
bos2019_df

,BibNumber,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,CountryOfResAbbrev,CountryOfResName,OfficialTime,RankOverall,RankOverGender
0,2,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,KEN,Kenya,2:07:57,1,1
1,6,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,ETH,Ethiopia,2:07:59,2,2
2,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,KEN,Kenya,2:08:07,3,3
3,8,"Kandie, Felix",32,M,Iten,NaN,NaN,KEN,Kenya,2:08:54,4,4
4,11,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,KEN,Kenya,2:08:55,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
26648,30798,"Shubert, Mackenzie",28,F,Malden,MA,Massachusetts,USA,United States of America,6:16:56,26636,11975
26649,25235,"Mcsweeney, Nikki",33,F,Natick,MA,Massachusetts,USA,United States of America,6:18:13,26637,11976
26650,25616,"Genest, Debbie",44,F,Harrisville,RI,Rhode Island,USA,United States of America,6:21:19,26640,11977
26651,23316,"Lewandowski, Denise",50,F,Boiling Springs,PA,Pennsylvania,USA,United States of America,6:22:27,26641,11978


In [14]:
# divide into two dataframes: US & other countries
usa_df = bos2019_df.loc[bos2019_df["CountryOfResAbbrev"] == "USA"]
world_df = bos2019_df.loc[bos2019_df["CountryOfResAbbrev"] != "USA"]
print(usa_df["BibNumber"].count())
print(world_df["BibNumber"].count())

19757
6784


In [15]:
# add address column for each dataframe (either city & state or city & country)
usa_df["Address"] = (usa_df["City"] + ", " + usa_df["StateAbbrev"])
world_df["Address"] = (world_df["City"] + ", " + world_df["CountryOfResName"])
world_df

/Users/lisamcgowan/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lisamcgowan/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,BibNumber,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,CountryOfResAbbrev,CountryOfResName,OfficialTime,RankOverall,RankOverGender,Address
0,2,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,KEN,Kenya,2:07:57,1,1,"Eldoret, Kenya"
1,6,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,ETH,Ethiopia,2:07:59,2,2,"Ambo, Ethiopia"
2,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,KEN,Kenya,2:08:07,3,3,"Eldoret, Kenya"
3,8,"Kandie, Felix",32,M,Iten,NaN,NaN,KEN,Kenya,2:08:54,4,4,"Iten, Kenya"
4,11,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,KEN,Kenya,2:08:55,5,5,"Keringet, Kenya"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26594,23225,"Perry, Debbie",58,F,Hertford,NaN,NaN,GBR,United Kingdom,6:04:39,26551,11921,"Hertford, United Kingdom"
26595,29848,"Perez, Annette",58,F,Guaynabo,PR,Puerto Rico,PRI,Puerto Rico,6:04:53,26552,11922,"Guaynabo, Puerto Rico"
26628,17988,"Cameron, Emma",37,F,Jabiru,NaN,NaN,AUS,Australia,6:11:20,26611,11955,"Jabiru, Australia"
26630,28538,"Kelkar, Rohini",59,F,Mumbai,NaN,NaN,IND,India,6:11:48,26614,11957,"Mumbai, India"


In [16]:
# get lists of unique addresses to cut down on api requests
# check lengths to see how many times Google api will be pinged
usa_address = usa_df["Address"].unique()
world_address = world_df["Address"].unique()
print(len(usa_address))
print(len(world_address))


4498
2737


In [17]:
# set up url to get coordinates
base_url = "https://maps.googleapis.com/maps/api/geocode/json"
params = { 
    "key": g_key,
}

# testing
#address = "Wandlitz, Germany"
#params["address"] = address
#response = requests.get(base_url, params=params).json()
#print(json.dumps(response, indent=4))

In [56]:
# testing 
#print(response["results"][0]["geometry"]["location"]["lat"])
#print(response["results"][0]["geometry"]["location"]["lng"])
#print(response["status"])

52.75422
13.47277
OK


In [18]:
# create lists to store coordinates
world_lat = []
world_lng = []

# make api requests for non-US addresses
for address in world_address:
    params["address"] = address
    response = requests.get(base_url, params=params).json()
    if response["status"] == "OK":
        world_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        world_lng.append(response["results"][0]["geometry"]["location"]["lng"])
    else:
        world_lat.append("none")
        world_lng.append("none")
    


In [19]:
# create dataframe for non-US coordinates
world_dict = {
    "Address": world_address,
    "Lat": world_lat,
    "Lng": world_lng
}
world_coord_df = pd.DataFrame(world_dict)
world_coord_df

,Address,Lat,Lng
0,"Eldoret, Kenya",0.514277,35.2698
1,"Ambo, Ethiopia",8.95806,37.9321
2,"Iten, Kenya",0.673235,35.5083
3,"Keringet, Kenya",-0.469779,36.0003
4,"Kaptagat, Kenya",0.436426,35.4512
...,...,...,...
2732,"Wandlitz, Germany",52.7542,13.4728
2733,"Andover, United Kingdom",51.2112,-1.49192
2734,"Aston Clinton, United Kingdom",51.7962,-0.727713
2735,"Hertford, United Kingdom",51.7958,-0.081157


In [20]:
# see count of "none"
world_coord_df["Lat"].value_counts()

none                 14
19.4326077            9
4.710988599999999     6
22.3193039            5
-34.6036844           5
                     ..
50.62925              1
56.1165227            1
25.776468             1
3.2617697             1
49.06475529999999     1
Name: Lat, Length: 2498, dtype: int64

In [22]:
# create dataframe for US coordinates
usa_lat = []
usa_lng = []
    
for address in usa_address:
    params["address"] = address
    response = requests.get(base_url, params=params).json()
    if response["status"] == "OK":
        usa_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        usa_lng.append(response["results"][0]["geometry"]["location"]["lng"])
    else:
        usa_lat.append("none")
        usa_lng.append("none")

In [23]:
# create dataframe for US coordinates
usa_dict = {
    "Address": usa_address,
    "Lat": usa_lat,
    "Lng": usa_lng
}
usa_coord_df = pd.DataFrame(usa_dict)
usa_coord_df

,Address,Lat,Lng
0,"Flagstaff, AZ",35.1983,-111.651
1,"Mapleton, UT",40.1302,-111.579
2,"Fountain, CO",38.6822,-104.701
3,"Colorado Springs, CO",38.8339,-104.821
4,"Tampa, FL",27.9506,-82.4572
...,...,...,...
4493,"Merritt Island, FL",28.3181,-80.666
4494,"Paintsville, KY",37.8145,-82.8071
4495,"Middleboro, MA",41.893,-70.9108
4496,"Marlboro, MA",42.3459,-71.5523


In [30]:
# create dataframe of all addresses
all_coords_df = pd.concat([world_coord_df, usa_coord_df])
all_coords_df

,Address,Lat,Lng
0,"Eldoret, Kenya",0.514277,35.2698
1,"Ambo, Ethiopia",8.95806,37.9321
2,"Iten, Kenya",0.673235,35.5083
3,"Keringet, Kenya",-0.469779,36.0003
4,"Kaptagat, Kenya",0.436426,35.4512
...,...,...,...
4493,"Merritt Island, FL",28.3181,-80.666
4494,"Paintsville, KY",37.8145,-82.8071
4495,"Middleboro, MA",41.893,-70.9108
4496,"Marlboro, MA",42.3459,-71.5523


In [28]:
# create dataframe of all data with addresses added
all_runners_df = pd.concat([world_df, usa_df])
all_runners_df

,BibNumber,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,CountryOfResAbbrev,CountryOfResName,OfficialTime,RankOverall,RankOverGender,Address
0,2,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,KEN,Kenya,2:07:57,1,1,"Eldoret, Kenya"
1,6,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,ETH,Ethiopia,2:07:59,2,2,"Ambo, Ethiopia"
2,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,KEN,Kenya,2:08:07,3,3,"Eldoret, Kenya"
3,8,"Kandie, Felix",32,M,Iten,NaN,NaN,KEN,Kenya,2:08:54,4,4,"Iten, Kenya"
4,11,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,KEN,Kenya,2:08:55,5,5,"Keringet, Kenya"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26648,30798,"Shubert, Mackenzie",28,F,Malden,MA,Massachusetts,USA,United States of America,6:16:56,26636,11975,"Malden, MA"
26649,25235,"Mcsweeney, Nikki",33,F,Natick,MA,Massachusetts,USA,United States of America,6:18:13,26637,11976,"Natick, MA"
26650,25616,"Genest, Debbie",44,F,Harrisville,RI,Rhode Island,USA,United States of America,6:21:19,26640,11977,"Harrisville, RI"
26651,23316,"Lewandowski, Denise",50,F,Boiling Springs,PA,Pennsylvania,USA,United States of America,6:22:27,26641,11978,"Boiling Springs, PA"


In [31]:
# merge coords into main data using "Address" column
merge_df = pd.merge(all_runners_df, all_coords_df, on="Address", how="left")
merge_df

,BibNumber,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,CountryOfResAbbrev,CountryOfResName,OfficialTime,RankOverall,RankOverGender,Address,Lat,Lng
0,2,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,KEN,Kenya,2:07:57,1,1,"Eldoret, Kenya",0.514277,35.2698
1,6,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,ETH,Ethiopia,2:07:59,2,2,"Ambo, Ethiopia",8.95806,37.9321
2,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,KEN,Kenya,2:08:07,3,3,"Eldoret, Kenya",0.514277,35.2698
3,8,"Kandie, Felix",32,M,Iten,NaN,NaN,KEN,Kenya,2:08:54,4,4,"Iten, Kenya",0.673235,35.5083
4,11,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,KEN,Kenya,2:08:55,5,5,"Keringet, Kenya",-0.469779,36.0003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26536,30798,"Shubert, Mackenzie",28,F,Malden,MA,Massachusetts,USA,United States of America,6:16:56,26636,11975,"Malden, MA",42.4251,-71.0662
26537,25235,"Mcsweeney, Nikki",33,F,Natick,MA,Massachusetts,USA,United States of America,6:18:13,26637,11976,"Natick, MA",42.2775,-71.3468
26538,25616,"Genest, Debbie",44,F,Harrisville,RI,Rhode Island,USA,United States of America,6:21:19,26640,11977,"Harrisville, RI",41.9657,-71.6745
26539,23316,"Lewandowski, Denise",50,F,Boiling Springs,PA,Pennsylvania,USA,United States of America,6:22:27,26641,11978,"Boiling Springs, PA",40.1498,-77.1283


In [33]:
# count how many lines have no coords
none_count = (merge_df["Lat"] == "none").sum()
none_count

24

In [34]:
# remove lines with no coords
boston2019_coords_df = merge_df.loc[(merge_df["Lat"] != "none")]
boston2019_coords_df

,BibNumber,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,CountryOfResAbbrev,CountryOfResName,OfficialTime,RankOverall,RankOverGender,Address,Lat,Lng
0,2,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,KEN,Kenya,2:07:57,1,1,"Eldoret, Kenya",0.514277,35.2698
1,6,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,ETH,Ethiopia,2:07:59,2,2,"Ambo, Ethiopia",8.95806,37.9321
2,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,KEN,Kenya,2:08:07,3,3,"Eldoret, Kenya",0.514277,35.2698
3,8,"Kandie, Felix",32,M,Iten,NaN,NaN,KEN,Kenya,2:08:54,4,4,"Iten, Kenya",0.673235,35.5083
4,11,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,KEN,Kenya,2:08:55,5,5,"Keringet, Kenya",-0.469779,36.0003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26536,30798,"Shubert, Mackenzie",28,F,Malden,MA,Massachusetts,USA,United States of America,6:16:56,26636,11975,"Malden, MA",42.4251,-71.0662
26537,25235,"Mcsweeney, Nikki",33,F,Natick,MA,Massachusetts,USA,United States of America,6:18:13,26637,11976,"Natick, MA",42.2775,-71.3468
26538,25616,"Genest, Debbie",44,F,Harrisville,RI,Rhode Island,USA,United States of America,6:21:19,26640,11977,"Harrisville, RI",41.9657,-71.6745
26539,23316,"Lewandowski, Denise",50,F,Boiling Springs,PA,Pennsylvania,USA,United States of America,6:22:27,26641,11978,"Boiling Springs, PA",40.1498,-77.1283


In [35]:
# create csv file of runner info & coords (just in case variables are cleared - don't want to have to request again!)
boston2019_coords_df.to_csv("output/2019boston_marathon_coords.csv", encoding="utf-8", index=True)

In [37]:
# find unique address list with coordinates to use to pull elevations
all_coords_df

,Address,Lat,Lng
0,"Eldoret, Kenya",0.514277,35.2698
1,"Ambo, Ethiopia",8.95806,37.9321
2,"Iten, Kenya",0.673235,35.5083
3,"Keringet, Kenya",-0.469779,36.0003
4,"Kaptagat, Kenya",0.436426,35.4512
...,...,...,...
4493,"Merritt Island, FL",28.3181,-80.666
4494,"Paintsville, KY",37.8145,-82.8071
4495,"Middleboro, MA",41.893,-70.9108
4496,"Marlboro, MA",42.3459,-71.5523


In [43]:
# create column that combines lat & lng
all_coords_df["Coordinates"]= all_coords_df["Lat"].astype(str) +", "+ all_coords_df["Lng"].astype(str)
all_coords_df

,Address,Lat,Lng,Coordinates
0,"Eldoret, Kenya",0.514277,35.2698,"0.5142774999999999, 35.2697802"
1,"Ambo, Ethiopia",8.95806,37.9321,"8.9580605, 37.9321008"
2,"Iten, Kenya",0.673235,35.5083,"0.6732353999999999, 35.5083136"
3,"Keringet, Kenya",-0.469779,36.0003,"-0.4697789, 36.0003237"
4,"Kaptagat, Kenya",0.436426,35.4512,"0.436426, 35.451241"
...,...,...,...,...
4493,"Merritt Island, FL",28.3181,-80.666,"28.3180688, -80.6659842"
4494,"Paintsville, KY",37.8145,-82.8071,"37.8145384, -82.8071054"
4495,"Middleboro, MA",41.893,-70.9108,"41.8929942, -70.9107708"
4496,"Marlboro, MA",42.3459,-71.5523,"42.3459271, -71.5522874"


In [49]:
# create list of coordinates
coord_list = all_coords_df["Coordinates"].tolist()
print(coord_list[0])

0.5142774999999999, 35.2697802


In [50]:
# set up url to get elevations
base_url = "https://maps.googleapis.com/maps/api/elevation/json"
params = { 
    "key": g_key,
}

# testing api
#locations = coord_list[0]
#params["locations"] = locations
#response = requests.get(base_url, params=params).json()
#print(json.dumps(response, indent=4))

{
    "results": [
        {
            "elevation": 2070.04833984375,
            "location": {
                "lat": 0.5142774999999999,
                "lng": 35.2697802
            },
            "resolution": 152.7032318115234
        }
    ],
    "status": "OK"
}


In [51]:
# testing api
#print(response["results"][0]["elevation"])
#print(response["status"])

2070.04833984375
OK


In [52]:
# create list to store elevations
elevation = []

# make api requests for elevations
for coord in coord_list:
    params["locations"] = coord
    response = requests.get(base_url, params=params).json()
    if response["status"] == "OK":
        elevation.append(response["results"][0]["elevation"])
    else:
        elevation.append("none")

In [56]:
# add elevations to the dataframe
all_coords_df["Elevation"] = elevation
all_coords_df = all_coords_df.reset_index(drop=True)
all_coords_df

,Address,Lat,Lng,Coordinates,Elevation
0,"Eldoret, Kenya",0.514277,35.2698,"0.5142774999999999, 35.2697802",2070.05
1,"Ambo, Ethiopia",8.95806,37.9321,"8.9580605, 37.9321008",2376.24
2,"Iten, Kenya",0.673235,35.5083,"0.6732353999999999, 35.5083136",2358.82
3,"Keringet, Kenya",-0.469779,36.0003,"-0.4697789, 36.0003237",2272.98
4,"Kaptagat, Kenya",0.436426,35.4512,"0.436426, 35.451241",2337.27
...,...,...,...,...,...
7230,"Merritt Island, FL",28.3181,-80.666,"28.3180688, -80.6659842",-0.316639
7231,"Paintsville, KY",37.8145,-82.8071,"37.8145384, -82.8071054",186.464
7232,"Middleboro, MA",41.893,-70.9108,"41.8929942, -70.9107708",31.5997
7233,"Marlboro, MA",42.3459,-71.5523,"42.3459271, -71.5522874",118.595


In [58]:
# merge elevations into main dataframe with all data
all_elevation_df = pd.merge(boston2019_coords_df, all_coords_df, on="Address", how="left")
all_elevation_df

,BibNumber,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,CountryOfResAbbrev,CountryOfResName,OfficialTime,RankOverall,RankOverGender,Address,Lat_x,Lng_x,Lat_y,Lng_y,Coordinates,Elevation
0,2,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,KEN,Kenya,2:07:57,1,1,"Eldoret, Kenya",0.514277,35.2698,0.514277,35.2698,"0.5142774999999999, 35.2697802",2070.05
1,6,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,ETH,Ethiopia,2:07:59,2,2,"Ambo, Ethiopia",8.95806,37.9321,8.95806,37.9321,"8.9580605, 37.9321008",2376.24
2,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,KEN,Kenya,2:08:07,3,3,"Eldoret, Kenya",0.514277,35.2698,0.514277,35.2698,"0.5142774999999999, 35.2697802",2070.05
3,8,"Kandie, Felix",32,M,Iten,NaN,NaN,KEN,Kenya,2:08:54,4,4,"Iten, Kenya",0.673235,35.5083,0.673235,35.5083,"0.6732353999999999, 35.5083136",2358.82
4,11,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,KEN,Kenya,2:08:55,5,5,"Keringet, Kenya",-0.469779,36.0003,-0.469779,36.0003,"-0.4697789, 36.0003237",2272.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26512,30798,"Shubert, Mackenzie",28,F,Malden,MA,Massachusetts,USA,United States of America,6:16:56,26636,11975,"Malden, MA",42.4251,-71.0662,42.4251,-71.0662,"42.4250964, -71.066163",3.17294
26513,25235,"Mcsweeney, Nikki",33,F,Natick,MA,Massachusetts,USA,United States of America,6:18:13,26637,11976,"Natick, MA",42.2775,-71.3468,42.2775,-71.3468,"42.2775281, -71.3468091",51.8774
26514,25616,"Genest, Debbie",44,F,Harrisville,RI,Rhode Island,USA,United States of America,6:21:19,26640,11977,"Harrisville, RI",41.9657,-71.6745,41.9657,-71.6745,"41.96565390000001, -71.6745112",97.6782
26515,23316,"Lewandowski, Denise",50,F,Boiling Springs,PA,Pennsylvania,USA,United States of America,6:22:27,26641,11978,"Boiling Springs, PA",40.1498,-77.1283,40.1498,-77.1283,"40.14981239999999, -77.1283133",147.603


In [59]:
# count elevations listed as "none"
none_elevation = (all_elevation_df["Elevation"] == "none").sum()
none_elevation

0

In [60]:
# remove duplicate lat & lng columns
elevation_df = all_elevation_df.drop(["Lat_y", "Lng_y"], axis=1)
elevation_df

,BibNumber,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,CountryOfResAbbrev,CountryOfResName,OfficialTime,RankOverall,RankOverGender,Address,Lat_x,Lng_x,Coordinates,Elevation
0,2,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,KEN,Kenya,2:07:57,1,1,"Eldoret, Kenya",0.514277,35.2698,"0.5142774999999999, 35.2697802",2070.05
1,6,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,ETH,Ethiopia,2:07:59,2,2,"Ambo, Ethiopia",8.95806,37.9321,"8.9580605, 37.9321008",2376.24
2,7,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,KEN,Kenya,2:08:07,3,3,"Eldoret, Kenya",0.514277,35.2698,"0.5142774999999999, 35.2697802",2070.05
3,8,"Kandie, Felix",32,M,Iten,NaN,NaN,KEN,Kenya,2:08:54,4,4,"Iten, Kenya",0.673235,35.5083,"0.6732353999999999, 35.5083136",2358.82
4,11,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,KEN,Kenya,2:08:55,5,5,"Keringet, Kenya",-0.469779,36.0003,"-0.4697789, 36.0003237",2272.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26512,30798,"Shubert, Mackenzie",28,F,Malden,MA,Massachusetts,USA,United States of America,6:16:56,26636,11975,"Malden, MA",42.4251,-71.0662,"42.4250964, -71.066163",3.17294
26513,25235,"Mcsweeney, Nikki",33,F,Natick,MA,Massachusetts,USA,United States of America,6:18:13,26637,11976,"Natick, MA",42.2775,-71.3468,"42.2775281, -71.3468091",51.8774
26514,25616,"Genest, Debbie",44,F,Harrisville,RI,Rhode Island,USA,United States of America,6:21:19,26640,11977,"Harrisville, RI",41.9657,-71.6745,"41.96565390000001, -71.6745112",97.6782
26515,23316,"Lewandowski, Denise",50,F,Boiling Springs,PA,Pennsylvania,USA,United States of America,6:22:27,26641,11978,"Boiling Springs, PA",40.1498,-77.1283,"40.14981239999999, -77.1283133",147.603


In [61]:
# create csv file of runner info, coords, & elevation (to use for analysis notebook)
elevation_df.to_csv("output/2019boston_marathon_elevation.csv", encoding="utf-8", index=True)